In [7]:
%pip install numpy
%pip install Pillow
%pip install opencv-python

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
  Using cached opencv_python-4.11.0.86-cp37-abi3-macosx_13_0_arm64.whl.metadata (20 kB)
Using cached opencv_python-4.11.0.86-cp37-abi3-macosx_13_0_arm64.whl (37.3 MB)
Note: you may need to restart the kernel to use updated packages.


In [8]:
import random
import numpy as np
from PIL import Image, ImageEnhance
import cv2

def random_crop(img: Image.Image, crop_size: tuple[int, int]) -> Image.Image:
    """Randomly crop the given PIL image to crop_size (width, height)."""
    w, h = img.size
    crop_w, crop_h = crop_size
    left = random.randint(0, w - crop_w)
    top = random.randint(0, h - crop_h)
    return img.crop((left, top, left + crop_w, top + crop_h))

def color_jitter_manual(
    img: Image.Image,
    brightness: float = 0.4,
    contrast: float = 0.4,
    saturation: float = 0.4
) -> Image.Image:
    """Apply manual color jitter using PIL's ImageEnhance."""
    # Brightness
    factor = 1 + random.uniform(-brightness, brightness)
    img = ImageEnhance.Brightness(img).enhance(factor)
    # Contrast
    factor = 1 + random.uniform(-contrast, contrast)
    img = ImageEnhance.Contrast(img).enhance(factor)
    # Saturation
    factor = 1 + random.uniform(-saturation, saturation)
    img = ImageEnhance.Color(img).enhance(factor)
    return img

def crop_color_jitter(
    img: Image.Image,
    crop_size: tuple[int, int] = (224, 224)
) -> Image.Image:
    """Random crop followed by color jitter."""
    cropped = random_crop(img, crop_size)
    return color_jitter_manual(cropped)

def sobel_filter(img: Image.Image) -> Image.Image:
    """Convert to grayscale and apply Sobel edge detection."""
    img_np = np.array(img.convert('RGB'))
    gray = cv2.cvtColor(img_np, cv2.COLOR_RGB2GRAY)
    sobelx = cv2.Sobel(gray, cv2.CV_64F, 1, 0, ksize=3)
    sobely = cv2.Sobel(gray, cv2.CV_64F, 0, 1, ksize=3)
    sobel = np.sqrt(sobelx**2 + sobely**2)
    sobel = (sobel / sobel.max() * 255).astype(np.uint8)
    return Image.fromarray(sobel)

def crop_sobel(
    img: Image.Image,
    crop_size: tuple[int, int] = (224, 224)
) -> Image.Image:
    """Random crop followed by Sobel edge detection."""
    cropped = random_crop(img, crop_size)
    return sobel_filter(cropped)

Perform the augmentations

In [ ]:
img = Image.open(input_path).convert('RGB')
aug1 = crop_color_jitter(img, crop_size)
aug2 = crop_sobel(img, crop_size)
aug1.save("crop_color.jpg")
aug2.save("crop_sobel.jpg")
print("Saved augmentations as 'crop_color.jpg' and 'crop_sobel.jpg'")
